In [3]:
import random
import math
from typing import List, Callable, Tuple

import pandas as pd

# 20 canonical amino acids (no X)
AMINO_ACIDS = list('ACDEFGHIKLMNPQRSTVWY')

def mutate_sequence(seq: str, n_mutations: int = 1) -> str:
    """Return a new sequence with *n* random point mutations **excluding padding 'X' positions**.
    If all positions are 'X', the original sequence is returned unmodified.
    """
    seq_list = list(seq)
    # Positions eligible for mutation (non‑padding)
    valid_positions = [i for i, aa in enumerate(seq_list) if aa != 'X']
    if not valid_positions:
        return seq  # nothing to mutate
    n_mutations = min(n_mutations, len(valid_positions))
    positions = random.sample(valid_positions, n_mutations)
    for pos in positions:
        original = seq_list[pos]
        choices = [aa for aa in AMINO_ACIDS if aa != original]
        seq_list[pos] = random.choice(choices)
    return ''.join(seq_list)

def hill_climb(seed: str,
               fitness_fn: Callable[[str], float],
               local_k2_samples: int = 1000,
               restarts: int = 100) -> List[str]:
    """Implements greedy HC as in Li et al.
    Args:
        seed: initial sequence
        fitness_fn: objective function; higher is better
        local_k2_samples: #random k=2 mutants to include in local neighbourhood
        restarts: number of random restarts (each with 2‑mutant perturbation)
    Returns a list of unique sequences visited across all restarts (excluding seed)."""
    best_sequences = set()
    for _ in range(restarts):
        current = mutate_sequence(seed, 2)  # expected k=2
        current_fit = fitness_fn(current)
        improved = True
        while improved:
            neighbourhood = []
            # all single mutants
            for i in range(len(current)):
                for aa in AMINO_ACIDS:
                    if aa != current[i]:
                        mutant = current[:i] + aa + current[i+1:]
                        neighbourhood.append(mutant)
            # random k=2 mutants
            for _ in range(local_k2_samples):
                neighbourhood.append(mutate_sequence(current, 2))
            # evaluate and pick best
            scored = [(fitness_fn(seq), seq) for seq in neighbourhood]
            best_fit, best_seq = max(scored, key=lambda x: x[0])
            if best_fit > current_fit:
                best_sequences.add(best_seq)
                current, current_fit = best_seq, best_fit
            else:
                improved = False
    return list(best_sequences)

def genetic_algorithm(seed_population: List[str],
                      fitness_fn: Callable[[str], float],
                      beta: float = 0.2,
                      p_mut: float = 0.05,
                      max_generations: int = 100,
                      population_size: int = 100) -> List[str]:
    """Evolutionary GA with Wright–Fisher style selection & single‑point crossover."""
    pop = seed_population[:population_size]
    all_offspring = set(pop)
    for _ in range(max_generations):
        fitnesses = [fitness_fn(s) for s in pop]
        probs = [math.exp(f/beta) for f in fitnesses]
        total = sum(probs)
        probs = [p/total for p in probs]
        parents = random.choices(pop, weights=probs, k=2*population_size)
        children = []
        for i in range(0, len(parents), 2):
            p1, p2 = parents[i], parents[i+1]
            cx_pt = random.randint(1, len(p1)-1)
            child = p1[:cx_pt] + p2[cx_pt:]
            # mutate child
            child_list = list(child)
            for j in range(len(child_list)):
                if random.random() < p_mut:
                    child_list[j] = random.choice([aa for aa in AMINO_ACIDS if aa != child_list[j]])
            child = ''.join(child_list)
            children.append(child)
        pop = children[:population_size]
        all_offspring.update(pop)
        # convergence criterion: diversity check
        if len(set(pop)) < 0.1 * population_size:
            break
    return list(all_offspring - set(seed_population))

def gibbs_sampling(seed: str,
                   fitness_fn: Callable[[str], float],
                   gamma: float = 20.0,
                   iterations: int = 30000) -> List[str]:
    """Random‑scan Gibbs sampler over sequence positions."""
    current = seed
    visited = set()
    L = len(seed)
    for _ in range(iterations):
        i = random.randrange(L)
        scores = []
        for aa in AMINO_ACIDS:
            if aa == current[i]:
                scores.append(-math.inf)
            else:
                mutant = current[:i] + aa + current[i+1:]
                scores.append(gamma*fitness_fn(mutant))
        # softmax sampling
        max_score = max(scores)
        probs = [math.exp(s-max_score) for s in scores]
        total = sum(probs)
        probs = [p/total for p in probs]
        new_aa = random.choices(AMINO_ACIDS, weights=probs, k=1)[0]
        if new_aa != current[i]:
            current = current[:i] + new_aa + current[i+1:]
            visited.add(current)
    return list(visited)


In [ ]:
# Load your data
# Suppose df has columns: 'sequence' and 'fitness'
df = pd.read_csv('input_sequences.json')

# Build a fitness lookup dict
fitness_lookup = dict(zip(df['sequence'], df['fitness']))

def fitness_fn(seq: str) -> float:
    # If seq already measured, return value; else assign a penalty (or predictive model)
    return fitness_lookup.get(seq, -1e9)

# --- Hill climb ---
hc_seqs = hill_climb(seed=df.loc[0, 'sequence'], fitness_fn=fitness_fn)

# --- Genetic algorithm ---
seed_pop = df['sequence'].head(10).tolist()
ga_seqs = genetic_algorithm(seed_pop, fitness_fn)

# --- Gibbs sampling ---
gs_seqs = gibbs_sampling(seed=df.loc[0, 'sequence'], fitness_fn=fitness_fn)

# Merge & save
new_seqs = pd.DataFrame({'sequence': list(set(hc_seqs+ga_seqs+gs_seqs))})
new_seqs.to_csv('suggested_sequences.csv', index=False)
print(f'Generated {len(new_seqs)} unique suggestions')


FileNotFoundError: [Errno 2] No such file or directory: 'input_sequences.csv'